In [ ]:
# Cài đặt các thư viện cần thiết
!pip install requests beautifulsoup4 pandas numpy torch
!pip install -U sentence-transformers faiss-cpu

In [ ]:
# Import các thư viện
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
from sentence_transformers import SentenceTransformer
import faiss
import pickle
from tqdm.notebook import tqdm

print("Tất cả thư viện đã được import thành công!")

In [ ]:
# Xây dựng bộ crawler dữ liệu từ Vinmec

def get_article_links(category_url):
    """Lấy tất cả link bài viết từ một trang danh mục của Vinmec."""
    links = []
    try:
        response = requests.get(category_url, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Tìm tất cả các thẻ 'a' có href chứa '/tin-tuc/' trong phần body của trang
            # Selector này cần được kiểm tra và cập nhật nếu cấu trúc web thay đổi
            articles = soup.select('div.body a[href*="/tin-tuc/"]')
            for article in articles:
                link = article.get('href')
                if link and not link.startswith('http'):
                    link = 'https://www.vinmec.com' + link
                if link not in links:
                    links.append(link)
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi truy cập {category_url}: {e}")
    return links

def scrape_article_content(article_url):
    """Crawl tiêu đề và nội dung chi tiết của một bài viết."""
    data = {'url': article_url, 'title': '', 'content': ''}
    try:
        response = requests.get(article_url, timeout=10)
        time.sleep(1) # Thêm độ trễ để tránh bị block
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Lấy tiêu đề bài viết
            title_tag = soup.find('h1')
            if title_tag:
                data['title'] = title_tag.get_text(strip=True)
            
            # Lấy nội dung chính của bài viết
            # Selector này thường chứa nội dung chính
            content_div = soup.find('div', class_='post_content')
            if content_div:
                # Loại bỏ các tag không cần thiết như script, style
                for tag in content_div(['script', 'style']):
                    tag.decompose()
                
                raw_text = content_div.get_text(separator='\n', strip=True)
                # Dọn dẹp text
                clean_text = re.sub(r'\n{2,}', '\n', raw_text) # Thay thế nhiều dòng trống bằng một
                data['content'] = clean_text
                
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi crawl bài viết {article_url}: {e}")
        
    return data

# === Thực thi crawling ===
# Ví dụ: Lấy các bài viết về bệnh Tiêu hóa
CATEGORY_URL = "https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/tieu-hoa-gan-mat/"
print(f"Bắt đầu crawl các link bài viết từ: {CATEGORY_URL}")
article_links = get_article_links(CATEGORY_URL)
print(f"Tìm thấy {len(article_links)} link bài viết.")

# Giới hạn số lượng bài viết để chạy demo nhanh
article_links = article_links[:20] 

all_articles_data = []
print("\nBắt đầu crawl nội dung chi tiết của từng bài viết...")
for link in tqdm(article_links):
    article_data = scrape_article_content(link)
    if article_data and article_data['title'] and article_data['content']:
        all_articles_data.append(article_data)

print(f"\nHoàn thành! Crawl được {len(all_articles_data)} bài viết có nội dung.")

# Chuyển thành DataFrame để dễ xử lý
df_health = pd.DataFrame(all_articles_data)
df_health.head()

In [ ]:
# Tiền xử lý và phân đoạn văn bản (Text Chunking)

def split_text_into_chunks(text, max_chunk_size=512, overlap=50):
    """
    Phân đoạn văn bản thành các chunks nhỏ hơn để vector hóa hiệu quả hơn.
    Mỗi chunk sẽ là một đơn vị kiến thức.
    """
    words = text.split()
    if not words:
        return []
        
    chunks = []
    current_chunk = []
    
    for word in words:
        current_chunk.append(word)
        if len(current_chunk) >= max_chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = current_chunk[-overlap:] # Giữ lại một phần overlap
            
    if current_chunk:
        chunks.append(" ".join(current_chunk))
        
    return chunks

# Tạo ra các đơn vị kiến thức (knowledge units)
knowledge_units = []
for index, row in df_health.iterrows():
    title = row['title']
    content = row['content']
    url = row['url']
    
    # Tạo một đoạn giới thiệu ban đầu
    intro_chunk = f"Tiêu đề: {title}. Nội dung: " + " ".join(content.split()[:100])
    knowledge_units.append({
        'url': url,
        'title': title,
        'chunk': intro_chunk
    })
    
    # Phân chia nội dung chính
    chunks = split_text_into_chunks(content)
    for chunk in chunks:
        knowledge_units.append({
            'url': url,
            'title': title,
            'chunk': chunk
        })

print(f"Tổng số đơn vị kiến thức (chunks) được tạo: {len(knowledge_units)}")

# Chuyển thành DataFrame
df_kb_health = pd.DataFrame(knowledge_units)
df_kb_health.head()

In [ ]:
# Vector hóa và xây dựng chỉ mục tìm kiếm với FAISS

# Sử dụng một model đã được pre-trained cho tiếng Việt
# 'bkai-foundation-models/vietnamese-bi-encoder' là một lựa chọn mạnh mẽ
print("Bắt đầu tải model Sentence Transformer...")
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')
print("Model đã tải xong.")

# Lấy danh sách các chunk text để vector hóa
chunks_to_encode = df_kb_health['chunk'].tolist()

# Bắt đầu quá trình encoding
print(f"Bắt đầu vector hóa {len(chunks_to_encode)} chunks văn bản...")
# show_progress_bar=True để theo dõi tiến trình
embeddings = model.encode(chunks_to_encode, show_progress_bar=True, normalize_embeddings=True)
print("Vector hóa hoàn tất!")
print(f"Shape của ma trận embeddings: {embeddings.shape}") # (số chunks, chiều vector)

# Xây dựng chỉ mục FAISS để tìm kiếm nhanh
dimension = embeddings.shape[1]  # Chiều của vector
index = faiss.IndexFlatL2(dimension) # Sử dụng L2 distance
index = faiss.IndexIDMap(index) # Map id của vector với vị trí của nó trong dataframe

# Thêm vectors vào chỉ mục
ids = np.array(range(len(chunks_to_encode))).astype('int64')
index.add_with_ids(embeddings, ids)

print(f"Đã thêm {index.ntotal} vector vào chỉ mục FAISS.")

# Lưu trữ Knowledge Base và chỉ mục FAISS
# 1. Lưu DataFrame chứa thông tin gốc
df_kb_health.to_csv('healthcare_kb.csv', index=False)

# 2. Lưu chỉ mục FAISS
with open('healthcare_faiss.index', 'wb') as f:
    faiss.write_index(index, faiss.StandardIOWriter(f))

print("Knowledge Base và chỉ mục FAISS đã được lưu thành công!")

In [ ]:
# Demo tìm kiếm thông tin trong Knowledge Base

# Load lại KB và chỉ mục (giả sử ở một phiên làm việc khác)
df_kb_loaded = pd.read_csv('healthcare_kb.csv')
with open('healthcare_faiss.index', 'rb') as f:
    index_loaded = faiss.read_index(faiss.StandardIOWriter(f))
    
model_loaded = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')


def search_health_info(query, top_k=3):
    """Hàm tìm kiếm thông tin y tế trong KB."""
    print(f"Truy vấn của bạn: '{query}'")
    
    # 1. Vector hóa câu truy vấn
    query_embedding = model_loaded.encode([query], normalize_embeddings=True)
    
    # 2. Tìm kiếm trong FAISS
    distances, ids = index_loaded.search(query_embedding, top_k)
    
    # 3. Lấy kết quả
    results = []
    print("\n--- Kết quả tìm kiếm phù hợp nhất ---")
    for i, doc_id in enumerate(ids[0]):
        if doc_id != -1: # FAISS trả về -1 nếu không có đủ kết quả
            result = {
                "score": 1 - distances[0][i], # Chuyển L2 distance sang score (càng gần 1 càng tốt)
                "title": df_kb_loaded.iloc[doc_id]['title'],
                "content_chunk": df_kb_loaded.iloc[doc_id]['chunk'],
                "url": df_kb_loaded.iloc[doc_id]['url']
            }
            results.append(result)
            print(f"\nKết quả {i+1} (Score: {result['score']:.4f})")
            print(f"   Tiêu đề: {result['title']}")
            print(f"   Nguồn: {result['url']}")
            print(f"   Nội dung liên quan: {result['content_chunk'][:500]}...")
            
    return results

# Thử tìm kiếm
search_health_info("triệu chứng của bệnh trào ngược dạ dày là gì?")
print("\n" + "="*50 + "\n")
search_health_info("cách chữa đau bụng do virus")